In [ ]:
import pickle as pkl
from sklearn.preprocessing import StandardScaler


with open('/content/drive/MyDrive/Dataset_without_memory/dataset.pickle', 'rb') as f:
    dataset = pkl.load(f)

with open('/content/drive/MyDrive/Dataset_without_memory/labels.pickle', 'rb') as f:
    labels = pkl.load(f)



labels = labels['1m class']
dataset = dataset.iloc[: , 1:]

import numpy as np
index1 = int(len(dataset) * 5/9)
index2 = int (len(dataset) * 6/9)


dataset_train , labels_train  = np.asarray(dataset[: index1]).astype('float32') , np.asarray(labels[:index1 ]).astype('float32')

dataset_val , labels_val = np.asarray(dataset[index1 : index2 ]).astype('float32') , np.asarray(labels[index1 : index2]).astype('float32')

dataset_test , labels_test = np.asarray(dataset[index2 :  ]).astype('float32') , np.asarray(labels[index2:]).astype('float32')



def standardization( dataframe1, dataframe2, dataframe3  ):
  ###we start the for loop form the second column because the first column is timesteps 
  for i in range (dataframe1.shape[1]):
    
    retrn_data = dataframe1[:, i].reshape(-1, 1)

    scaler = StandardScaler()
    retrn_data_standardized = scaler.fit_transform(retrn_data)

    dataframe1[: , i] = retrn_data_standardized.flatten()


##########validation data
    retrn_data = dataframe2[:, i].reshape(-1, 1)

    retrn_data_standardized = scaler.transform(retrn_data)

    dataframe2[: , i] = retrn_data_standardized.flatten()

##########test data

    retrn_data = dataframe3[:, i].reshape(-1, 1)

    retrn_data_standardized = scaler.transform(retrn_data)

    dataframe3[: , i] = retrn_data_standardized.flatten()




  return dataframe1 , dataframe2, dataframe3

X_train , X_val , X_test = standardization(dataset_train , dataset_val , dataset_test)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.utils.class_weight import compute_class_weight



class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique ( np.reshape( labels_train , (np.shape(labels_train)[0]))),
                                        y = np.reshape(labels_train ,(np.shape(labels_train)[0]))                                                 
                                    )
class_weights = dict(zip(np.unique(np.reshape(labels_train ,(np.shape(labels_train)[0]))), class_weights))
class_weights


{0.0: 1.173172355676699, 1.0: 0.8713759156801533}

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Create the logistic regression model
logreg = LogisticRegression(class_weight=class_weights)

# Train the model using the training data
logreg.fit(X_train, labels_train.ravel())

# Use the model to make predictions on the validation data
y_pred = logreg.predict(X_val)

# Calculate the accuracy of the model on the validation data
accuracy = accuracy_score(labels_val, y_pred)

# Print the accuracy
print("Accuracy on validation data:", accuracy)

# Use the model to make predictions on the test data
y_pred_test = logreg.predict(X_test)

# Calculate the accuracy of the model on the test data
accuracy_test = accuracy_score(labels_test, y_pred_test)

# Print the accuracy
print("Accuracy on test data:", accuracy_test)



Accuracy on validation data: 0.49280599463497904
Accuracy on test data: 0.481569612769731
